# Terrarium Colab Runner
Run this notebook to execute the Meeting Scheduling simulation in Colab using vLLM.

In [ ]:
# 1. Clone Repository
!rm -rf /content/terrarium
!git clone https://github.com/ylmzelff/terrarium.git
%cd terrarium

In [ ]:
# 2. Install Dependencies
!pip install -q -e .
!pip install vllm

In [ ]:
# 3. Check GPU
!nvidia-smi

In [ ]:
# 4. Patch Code (Disable PersonalAssistant / CoLLAB)
!python scripts/patch_for_colab.py

In [ ]:
# 5. Run Simulation
!python examples/base_main.py --config examples/configs/meeting_scheduling.yaml

In [ ]:
# 6. Verify Log (Check for Availability Table)
import os
from pathlib import Path

log_dirs = sorted(Path('logs/MeetingSchedulingEnvironment').glob('*'), key=os.path.getmtime, reverse=True)
if log_dirs:
    latest_log = log_dirs[0]
    blackboard_file = latest_log / 'blackboard_0.txt'
    if blackboard_file.exists():
        with open(blackboard_file, 'r') as f:
            content = f.read()
        if "AVAILABILITY TABLE" in content:
            print("✓ AVAILABILITY TABLE FOUND!")
            print("\n" + content)
        else:
            print("✗ Availability table not found in blackboard")
    else:
        print(f"✗ Blackboard file not found: {blackboard_file}")
else:
    print("✗ No log directories found")